In [1]:
import torch
from torch.cuda.amp import autocast, GradScaler
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Subset, ConcatDataset, Dataset
from torchvision import datasets, transforms, models
import mxnet as mx
from mxnet import recordio
import onnxruntime as ort
import os
import numpy as np
from tqdm.notebook import tqdm
from PIL import Image
from collections import defaultdict
import warnings
warnings.filterwarnings("ignore", message=".*VerifyOutputSizes.*")

In [28]:
train_transform = transforms.Compose([
    transforms.Resize((112, 112)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.05),
    transforms.RandomRotation(degrees=10), 
    transforms.RandomAffine(degrees=0, translate=(0.05, 0.05), scale=(0.95, 1.05)), 
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((112, 112)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [3]:
rec_path = './faces_vgg_112x112/train.rec'
idx_path = './faces_vgg_112x112/train.idx'

In [4]:
class MXNetRecordDataset(Dataset):
    def __init__(self, rec_path, idx_path, transform=None):
        self.record = recordio.MXIndexedRecordIO(idx_path, rec_path, 'r')
        self.keys = list(self.record.keys)
        self.transform = transform

    def set_transform(self, transform):
        self.transform = transform

    def __len__(self):
        return len(self.keys)

    def __getitem__(self, idx):
        # Loop sampai dapat sample valid
        for _ in range(len(self.keys)):
            key = self.keys[idx]
            item = self.record.read_idx(key)

            if item is None:
                # Ganti index → lanjut ke sample berikut
                idx = (idx + 1) % len(self)
                continue

            header, img_encoded = recordio.unpack(item)
            if img_encoded is None or len(img_encoded) == 0:
                # Ganti index → lanjut ke sample berikut
                idx = (idx + 1) % len(self)
                continue

            try:
                img = mx.image.imdecode(img_encoded)
            except Exception:
                # Kalau decoding gagal, skip
                idx = (idx + 1) % len(self)
                continue

            img = Image.fromarray(img.asnumpy())
            label = int(header.label)

            if self.transform:
                img = self.transform(img)

            return img, label

        # Kalau semua gagal (harusnya jarang banget)
        raise RuntimeError("Semua sample tidak valid dalam .rec file")

In [5]:
class TransformableDataset(Dataset):
    def __init__(self, base_dataset, transform=None):
        self.base_dataset = base_dataset
        self.transform = transform

    def __getitem__(self, idx):
        img, label = self.base_dataset[idx]
        if self.transform:
            img = self.transform(img)
        return img, label

    def __len__(self):
        return len(self.base_dataset)

    def set_transform(self, transform):
        self.transform = transform

In [6]:
def get_train_val_datasets(rec_path, idx_path, val_split=0.1, seed=42, max_samples=None, cache_path=None):
    full_dataset = MXNetRecordDataset(rec_path, idx_path)  # original dataset
    g = torch.Generator().manual_seed(seed)

    # coba load cache
    if cache_path:
        try:
            final_indices = np.load(cache_path)
            print(f"Load cached indices from {cache_path}")
        except FileNotFoundError:
            final_indices = None
    else:
        final_indices = None

    if max_samples is not None and final_indices is None:
        indices_per_label = defaultdict(list)

        # buka .rec/.idx menggunakan MXIndexedRecordIO
        recordio = mx.recordio.MXIndexedRecordIO(idx_path, rec_path, 'r')
        keys = list(recordio.keys)
        
        for i, key in enumerate(keys):
            header, _ = mx.recordio.unpack(recordio.read_idx(key))
            label = int(header.label) if isinstance(header.label, float) else int(header.label[0])
            indices_per_label[label].append(i)
            if (i+1) % 100000 == 0:
                print(f"Processed {i+1}/{len(keys)} records")

        # sampling per label
        final_indices = []
        for label, indices in indices_per_label.items():
            indices = torch.tensor(indices)
            if len(indices) > max_samples:
                perm = torch.randperm(len(indices), generator=g)[:max_samples]
                indices = indices[perm]
            final_indices.extend(indices.tolist())

        if cache_path:
            np.save(cache_path, final_indices)
            print(f"Saved cached indices to {cache_path}")

    if final_indices is not None:
        full_dataset = Subset(full_dataset, final_indices)

    full_dataset = TransformableDataset(full_dataset)

    # split train/val
    num_val = int(val_split * len(full_dataset))
    num_train = len(full_dataset) - num_val

    train_dataset, val_dataset = torch.utils.data.random_split(
        full_dataset,
        [num_train, num_val],
        generator=g
    )
    return train_dataset, val_dataset

In [29]:
train_dataset, val_dataset = get_train_val_datasets(rec_path, idx_path, val_split=0.2, seed=42, max_samples=25)

Processed 100000/3146439 records
Processed 200000/3146439 records
Processed 300000/3146439 records
Processed 400000/3146439 records
Processed 500000/3146439 records
Processed 600000/3146439 records
Processed 700000/3146439 records
Processed 800000/3146439 records
Processed 900000/3146439 records
Processed 1000000/3146439 records
Processed 1100000/3146439 records
Processed 1200000/3146439 records
Processed 1300000/3146439 records
Processed 1400000/3146439 records
Processed 1500000/3146439 records
Processed 1600000/3146439 records
Processed 1700000/3146439 records
Processed 1800000/3146439 records
Processed 1900000/3146439 records
Processed 2000000/3146439 records
Processed 2100000/3146439 records
Processed 2200000/3146439 records
Processed 2300000/3146439 records
Processed 2400000/3146439 records
Processed 2500000/3146439 records
Processed 2600000/3146439 records
Processed 2700000/3146439 records
Processed 2800000/3146439 records
Processed 2900000/3146439 records
Processed 3000000/31464

In [30]:
train_dataset.dataset.set_transform(train_transform)
val_dataset.dataset.set_transform(val_transform)

In [31]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4, pin_memory=True, persistent_workers=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4, pin_memory=True, persistent_workers=True)

In [18]:
so = ort.SessionOptions()
so.log_severity_level = 3
teacher_path = './glintr100.onnx'
ort_session = ort.InferenceSession(teacher_path, providers=['CUDAExecutionProvider', 'CPUExecutionProvider'], sess_options=so)

In [32]:
def get_teacher_embedding(images):
    resized = F.interpolate(images, size=(112, 112), mode='bilinear', align_corners=False)
    ort_inputs = {ort_session.get_inputs()[0].name: resized.cpu().numpy()}
    ort_outs = ort_session.run(None, ort_inputs)
    embedding = torch.tensor(ort_outs[0]).to(images.device)
    return F.normalize(embedding, p=2, dim=1)

In [33]:
class StudentResNet18(nn.Module):
    def __init__(self, embedding_size=512, dropout=0.2):
        super(StudentResNet18, self).__init__()
        base = models.resnet18(weights='DEFAULT')
        base.fc = nn.Identity()
        self.backbone = base
        self.bn = nn.BatchNorm1d(512)
        self.dropout = nn.Dropout(p=dropout)
        self.embedding = nn.Linear(512, embedding_size)
        nn.init.xavier_normal_(self.embedding.weight)
        nn.init.constant_(self.embedding.bias, 0)

    def forward(self, x):
        features = self.backbone(x)
        features = self.bn(features)  
        features = self.dropout(features)
        embeddings = self.embedding(features)
        embeddings = F.normalize(embeddings, p=2, dim=1, eps=1e-8)
        return embeddings

In [35]:
class CombinedLoss(nn.Module):
    def __init__(self, alpha=0.7, beta=0.3, temperature=4.0):
        super(CombinedLoss, self).__init__()
        self.alpha = alpha
        self.beta = beta
        self.temperature = temperature
        self.cosine_loss = nn.CosineEmbeddingLoss(margin=0.0)
        self.mse_loss = nn.MSELoss()
        
    def forward(self, student_emb, teacher_emb):
        batch_size = student_emb.size(0)
        target = torch.ones(batch_size).to(student_emb.device)\
        
        cos_loss = self.cosine_loss(student_emb, teacher_emb, target)
        mse_loss = self.mse_loss(student_emb / self.temperature, teacher_emb / self.temperature)
        
        total_loss = self.alpha * cos_loss + self.beta * mse_loss
        return total_loss, cos_loss, mse_loss

In [36]:
def train(student, device, train_loader, val_loader, optimizer, criterion,
          epochs=15, resume=False, ckpt_path="checkpoint.pth"):
    best_val_cos_sim = 0.0 
    patience, patience_counter = 5, 0 
    start_epoch = 0

    train_losses, val_losses = [], []
    train_cos_sims, val_cos_sims = [], []
    train_l2s, val_l2s = [], []

    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', factor=0.5, patience=3,
        threshold=0.001, min_lr=1e-7
    )

    # Mixed precision scaler
    scaler = GradScaler()

    if resume and os.path.exists(ckpt_path):
        checkpoint = torch.load(ckpt_path, map_location=device)
        student.load_state_dict(checkpoint["model_state_dict"])
        optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
        scheduler.load_state_dict(checkpoint["scheduler_state_dict"])
        start_epoch = checkpoint["epoch"] + 1
        best_val_cos_sim = checkpoint.get("best_val_cos_sim", 0.0)
        print(f"Resumed training from epoch {start_epoch} (best val cos sim: {best_val_cos_sim:.4f})")

    for epoch in range(start_epoch, epochs):
        student.train()
        train_loss, cos_sim_total, l2_total = 0, 0, 0
        train_cos_loss_total, train_mse_loss_total = 0, 0

        for images, _ in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} - Training"):
            images = images.to(device)

            optimizer.zero_grad()

            with autocast():
                stud_emb = student(images)
                with torch.no_grad():
                    teach_emb = get_teacher_embedding(images).to(device)

                total_loss, cos_loss, mse_loss = criterion(stud_emb, teach_emb)

            # Backward pass dengan gradient scaling
            scaler.scale(total_loss).backward()
            
            # Gradient clipping untuk stabilitas
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(student.parameters(), max_norm=1.0)
            
            scaler.step(optimizer)
            scaler.update()

            # Metrics tracking
            train_loss += total_loss.item()
            train_cos_loss_total += cos_loss.item()
            train_mse_loss_total += mse_loss.item()
            cos_sim_total += F.cosine_similarity(stud_emb, teach_emb, dim=1).mean().item()
            l2_total += torch.norm(stud_emb - teach_emb, dim=1).mean().item()

        # Average training metrics
        avg_train_loss = train_loss / len(train_loader)
        avg_train_cos = cos_sim_total / len(train_loader)
        avg_train_l2 = l2_total / len(train_loader)
        avg_train_cos_loss = train_cos_loss_total / len(train_loader)
        avg_train_mse_loss = train_mse_loss_total / len(train_loader)

        train_losses.append(avg_train_loss)
        train_cos_sims.append(avg_train_cos)
        train_l2s.append(avg_train_l2)

        # Validation
        student.eval()
        val_loss, cos_sim_total, l2_total = 0, 0, 0
        val_cos_loss_total, val_mse_loss_total = 0, 0
        
        with torch.no_grad():
            for images, _ in tqdm(val_loader, desc=f"Epoch {epoch+1}/{epochs} - Validation"):
                images = images.to(device)

                with autocast():
                    stud_emb = student(images)
                    teach_emb = get_teacher_embedding(images).to(device)
                    total_loss, cos_loss, mse_loss = criterion(stud_emb, teach_emb)

                val_loss += total_loss.item()
                val_cos_loss_total += cos_loss.item()
                val_mse_loss_total += mse_loss.item()
                cos_sim_total += F.cosine_similarity(stud_emb, teach_emb, dim=1).mean().item()
                l2_total += torch.norm(stud_emb - teach_emb, dim=1).mean().item()

        # Average validation metrics
        avg_val_loss = val_loss / len(val_loader)
        avg_val_cos = cos_sim_total / len(val_loader)
        avg_val_l2 = l2_total / len(val_loader)
        avg_val_cos_loss = val_cos_loss_total / len(val_loader)
        avg_val_mse_loss = val_mse_loss_total / len(val_loader)

        val_losses.append(avg_val_loss)
        val_cos_sims.append(avg_val_cos)
        val_l2s.append(avg_val_l2)

        print(f"Epoch {epoch+1:02d} | "
              f"Train: Loss={avg_train_loss:.4f}, CosSim={avg_train_cos:.4f}, L2={avg_train_l2:.4f}, "
              f"CosLoss={avg_train_cos_loss:.4f}, MSELoss={avg_train_mse_loss:.4f}")
        print(f"       | "
              f"Val: Loss={avg_val_loss:.4f}, CosSim={avg_val_cos:.4f}, L2={avg_val_l2:.4f}, "
              f"CosLoss={avg_val_cos_loss:.4f}, MSELoss={avg_val_mse_loss:.4f}")

        # Learning rate scheduling berdasarkan validation cosine similarity
        scheduler.step(avg_val_cos)

        # Early stopping berdasarkan cosine similarity
        if avg_val_cos > best_val_cos_sim:
            best_val_cos_sim = avg_val_cos
            torch.save(student.state_dict(), 'best_student.pth')
            print(f"*** New best model saved! Cosine Similarity: {best_val_cos_sim:.4f} ***")
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping triggered at epoch {epoch+1}")
                print(f"Best validation cosine similarity: {best_val_cos_sim:.4f}")
                break

        # Save checkpoint
        torch.save({
            "epoch": epoch,
            "model_state_dict": student.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "scheduler_state_dict": scheduler.state_dict(),
            "best_val_cos_sim": best_val_cos_sim,
        }, ckpt_path)

    return train_losses, val_losses, train_cos_sims, val_cos_sims, train_l2s, val_l2s

In [14]:
print("Active provider:", ort_session.get_providers())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

Active provider: ['CUDAExecutionProvider', 'CPUExecutionProvider']
cuda


In [37]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
student = StudentResNet18(embedding_size=512, dropout=0.2).to(device)
criterion = CombinedLoss(alpha=0.7, beta=0.3, temperature=4.0)
optimizer = optim.AdamW(student.parameters(), lr=5e-5, weight_decay=1e-4, betas=(0.9, 0.999), eps=1e-8)

results = train(student, device, train_loader, val_loader, optimizer, criterion, epochs=15, resume=False)

/tmp/ipykernel_4261/2001459881.py:17: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


Epoch 1/15 - Training:   0%|          | 0/2805 [00:00<?, ?it/s]

/tmp/ipykernel_4261/2001459881.py:38: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch 1/15 - Validation:   0%|          | 0/702 [00:00<?, ?it/s]

/tmp/ipykernel_4261/2001459881.py:82: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


...

Epoch 14/15 - Validation:   0%|          | 0/702 [00:00<?, ?it/s]

Epoch 14 | Train: Loss=0.2307, CosSim=0.6705, L2=0.7947, CosLoss=0.3295, MSELoss=0.0001
       | Val: Loss=0.2235, CosSim=0.6807, L2=0.7791, CosLoss=0.3193, MSELoss=0.0001
*** New best model saved! Cosine Similarity: 0.6807 ***


Epoch 15/15 - Training:   0%|          | 0/2805 [00:00<?, ?it/s]

Epoch 15/15 - Validation:   0%|          | 0/702 [00:00<?, ?it/s]

Epoch 15 | Train: Loss=0.2254, CosSim=0.6781, L2=0.7868, CosLoss=0.3219, MSELoss=0.0001
       | Val: Loss=0.2207, CosSim=0.6847, L2=0.7750, CosLoss=0.3153, MSELoss=0.0001
*** New best model saved! Cosine Similarity: 0.6847 ***


In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
student = StudentResNet18(embedding_size=512, dropout=0.3).to(device)
criterion_cos = nn.CosineEmbeddingLoss()
criterion_mse = nn.MSELoss()
optimizer = optim.Adam(student.parameters(), lr=1e-4, weight_decay=5e-4)
ckpt_path = './checkpoint.pth'

results = train(student, device, train_loader, val_loader, optimizer, criterion_cos, criterion_mse, epochs=15, resume=True, ckpt_path=ckpt_path)

Resumed training from epoch 9 (best val loss: 0.1300)


/tmp/ipykernel_9846/2064167286.py:14: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


Epoch 10/15 - Training:   0%|          | 0/2805 [00:00<?, ?it/s]

/tmp/ipykernel_9846/2064167286.py:34: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():  # <<< Mixed precision aktif di sini


Epoch 10/15 - Validation:   0%|          | 0/702 [00:00<?, ?it/s]

...

Epoch 13/15 - Validation:   0%|          | 0/702 [00:00<?, ?it/s]

Epoch 12 | Train Loss: 0.1104, CosSim: 0.7800, L2: 0.6491 || Val Loss: 0.1292, CosSim: 0.7427, L2: 0.7006


Epoch 14/15 - Training:   0%|          | 0/2805 [00:00<?, ?it/s]

Epoch 14/15 - Validation:   0%|          | 0/702 [00:00<?, ?it/s]

Epoch 13 | Train Loss: 0.1164, CosSim: 0.7681, L2: 0.6666 || Val Loss: 0.1345, CosSim: 0.7321, L2: 0.7153
Early stopping triggered
